# Reddit Scraper

## Import 

In [1]:
import praw
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import psaw as ps
from dotenv import load_dotenv
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
from tqdm import tqdm
from psaw import PushshiftAPI
from praw.exceptions import APIException


In [2]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [3]:
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [4]:
for post in reddit.subreddit("CryptoCurrency").search("Bitcoin", sort="new", limit=5):
    print(post.title, post.created_utc)


With AI here, the 4 year cycle will become obsolete. 1740571095.0
China's Supreme People's Court Met To Discuss The Legal Treatment of Crypto This Week - Bitcoin (BTC) Pivot Incoming 1740566036.0
Cathie Wood Says Bitcoin Consolidation Extremely Healthy Amid ‘Early Stages’ of Institutional Adoption 1740546195.0
Could Jack Dorsey actually be Satoshi Nakamoto? 1740534340.0
Steady Lads: Bitcoin Still On Track to Hit $200K This Year, Say Analysts - Decrypt 1740522323.0


Cryptos und Subreddits 

In [ ]:
crypto_terms = {
    # 🔹 Top Coins
    "Bitcoin": ["bitcoin", "btc", "sats", "bitcoin network"],
    "Ethereum": ["ethereum", "eth", "ether", "ethereum 2.0", "eth 2.0"],
    "Wrapped Ethereum": ["wrapped ethereum", "weth"],
    "Solana": ["solana", "sol", "sol coin"],
    "Avalanche": ["avalanche", "avax"],
    "Polkadot": ["polkadot", "dot"],
    "Near Protocol": ["near protocol", "near"],
    "Polygon": ["polygon", "matic"],
    "XRP": ["xrp", "ripple"],
    "Cardano": ["cardano", "ada"],
    "Cronos": ["cronos", "cro"],
    "Vulcan Forged PYR": ["vulcan forged", "pyr"],
    "Chiliz": ["chiliz", "chz"],
    "Illuvium": ["illuvium", "ilv"],
    "Ronin": ["ronin", "ron"],
    "Band Protocol": ["band protocol", "band"],
    "Optimism": ["optimism", "op"],
    "Celestia": ["celestia", "tia"],
    "Numerai": ["numerai", "nmr"],
    "Aethir": ["aethir", "ath"],
    "Sui": ["sui"],
    "Hyperliquid": ["hyperliquid", "hyp"],
    "Robinhood Coin": ["robinhood", "hood"],
    "Trump Coin": ["trump coin"],
    "USD Coin": ["usd coin", "usdc"],
    "Binance Coin": ["binance", "bnb"],
    "Litecoin": ["litecoin", "ltc"],
    "Dogecoin": ["dogecoin", "doge"],
    "Tron": ["tron", "trx"],
    "Aave": ["aave"],
    "Hedera": ["hedera", "hbar"],
    "Filecoin": ["filecoin", "fil"],
    "Cosmos": ["cosmos", "atom"],
    "Gala": ["gala"],
    "The Sandbox": ["sandbox", "sand"],
    "Audius": ["audius", "audio"],
    "Render": ["render", "rndr"],
    "Kusama": ["kusama", "ksm"],
    "VeChain": ["vechain", "vet"],
    "Chainlink": ["chainlink", "link"],
    "Berachain": ["berachain", "bera"],
    "TestCoin": ["testcoin", "test"],

    # 🔹 Meme-Coins
    "Shiba Inu": ["shiba inu", "shib"],
    "Pepe": ["pepe"],
    "Floki Inu": ["floki inu", "floki"],
    "Bonk": ["bonk"],
    "Wojak": ["wojak"],
    "Mog Coin": ["mog"],
    "Doge Killer (Leash)": ["leash"],
    "Baby Doge Coin": ["baby doge", "babydoge"],
    "Degen": ["degen"],
    "Toshi": ["toshi"],
    "Fartcoin": ["fartcoin"],
    "Banana": ["banana"],
    "Kabosu": ["kabosu"],
    "Husky": ["husky"],
    "Samoyedcoin": ["samoyedcoin", "samo"],
    "Milkbag": ["milkbag"],

    # 🔹 New Coins
    "Arbitrum": ["arbitrum", "arb"],
    "Starknet": ["starknet", "strk"],
    "Injective Protocol": ["injective", "inj"],
    "Sei Network": ["sei"],
    "Aptos": ["aptos", "apt"],
    "EigenLayer": ["eigenlayer", "eigen"],
    "Mantle": ["mantle", "mnt"],
    "Immutable X": ["immutable x", "imx"],
    "Ondo Finance": ["ondo"],
    "Worldcoin": ["worldcoin", "wld"],
    "Aerodrome": ["aerodrome", "aero"],
    "Jupiter": ["jupiter", "jup"],
    "THORChain": ["thorchain", "rune"],
    "Pendle": ["pendle"],
    "Kujira": ["kujira", "kuji"],
    "Noble": ["noble"],
    "Stride": ["stride", "strd"],
    "Dymension": ["dymension", "dym"],
    "Seamless Protocol": ["seamless", "seam"],
    "Blast": ["blast"],
    "Merlin": ["merlin"],
    "Tapioca": ["tapioca"],
    "Arcadia Finance": ["arcadia"],
    "Notcoin": ["notcoin", "not"],
    "Omni Network": ["omni"],
    "LayerZero": ["layerzero", "lz"],
    "ZetaChain": ["zetachain", "zeta"],
    "Friend.tech": ["friendtech"]
}


In [6]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance",        # Diskussionen über die Binance-Plattform  
    "Bitcoin",
    "ethtrader"
]

## Scraping 

Scraping Funktionen

In [7]:
  # 🔹 Funktion zum Scrapen von Posts und Kommentaren mit Backoff
def scrape_reddit(start_date, end_date, mode="initial"):
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())

    posts = []
    comments = []
    post_ids = set()
    request_count = 0  # Zählt die Anzahl der Requests

    for crypto_name, search_terms in crypto_terms.items():
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"🔍 Suche nach {crypto_name} in r/{subreddit_name}...")

            # 🟢 Suche nach allen relevanten Begriffen mit `.search()`
            for search_term in search_terms:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        created_dt = datetime.utcfromtimestamp(post.created_utc)

                        posts.append({
                            'post_id': post.id,
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'title': post.title,
                            'author': str(post.author),
                            'date': created_dt.date().isoformat(),
                            'time': created_dt.time().isoformat(),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"✅ Post gefunden: {post.title} (Suchbegriff: {search_term})")

                        # 🟢 Kommentare sammeln (mit Rate-Limit-Schutz)
                        try:
                            post.comments.replace_more(limit=0)
                            for comment in post.comments.list():
                                created_dt = datetime.utcfromtimestamp(comment.created_utc)
                                comments.append({
                                    'post_id': post.id,
                                    'comment_id': comment.id,
                                    'author': str(comment.author),
                                    'date': created_dt.date().isoformat(),
                                    'time': created_dt.time().isoformat(),
                                    'score': comment.score,
                                    'selftext': comment.body
                                })
                        except praw.exceptions.APIException as e:
                            if "RATELIMIT" in str(e):
                                print(f"⚠️ Reddit API-Limit erreicht. Warte 60 Sekunden...")
                                time.sleep(60)  # Wartezeit erhöhen
                            else:
                                print(f"⚠️ Fehler beim Abrufen der Kommentare: {e}")

                    # 🔄 Nach jeder `post.comments.list()` Anfrage prüfen, ob eine Pause nötig ist
                    request_count += 1
                    if request_count % 50 == 0:  # Nach 50 Requests eine kurze Pause
                        wait_time = 10  # Standard-Wartezeit
                        print(f"⏳ Warte {wait_time} Sekunden, um Rate-Limit zu vermeiden...")
                        time.sleep(wait_time)

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    print(f"✅ Scrape abgeschlossen: {len(df_posts)} Posts & {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# 🔹 Starte den Scraper für die letzten 3 Monate
start_of_period = datetime(2024, 11, 1)  # Startzeitpunkt
now = datetime.now()  # Aktueller Zeitpunkt
print("🚀 Starte den Scraper für die letzten 3 Monate...")
df_posts, df_comments = scrape_reddit(start_of_period, now)

print("✅ Daten erfolgreich gespeichert & bereit für weitere Analysen.")


🚀 Starte den Scraper für die letzten 3 Monate...
🔍 Suche nach Bitcoin in r/CryptoCurrency...
✅ Post gefunden: With AI here, the 4 year cycle will become obsolete. (Suchbegriff: bitcoin)
✅ Post gefunden: China's Supreme People's Court Met To Discuss The Legal Treatment of Crypto This Week - Bitcoin (BTC) Pivot Incoming (Suchbegriff: bitcoin)
✅ Post gefunden: Cathie Wood Says Bitcoin Consolidation Extremely Healthy Amid ‘Early Stages’ of Institutional Adoption (Suchbegriff: bitcoin)
✅ Post gefunden: Could Jack Dorsey actually be Satoshi Nakamoto? (Suchbegriff: bitcoin)
✅ Post gefunden: Steady Lads: Bitcoin Still On Track to Hit $200K This Year, Say Analysts - Decrypt (Suchbegriff: bitcoin)
✅ Post gefunden: A little hopium for all of us. M2 money supply is closely correlated with Bitcoin.. and here's what M2 is doing as of today (Suchbegriff: bitcoin)
✅ Post gefunden: Peter Schiff Rips Michael Saylor’s Bitcoin Boldness, Slams Strategy and BTC Investors (Suchbegriff: bitcoin)
✅ Post gefund

## Sentiment

In [8]:
# 🔹 GPU nutzen, falls verfügbar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Verwende Gerät: {device}")

# 🔹 CryptoBERT-Modell laden
MODEL_NAME = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
model.eval()  # Setzt das Modell in den Evaluationsmodus

# 🔹 Funktion zur Sentiment-Analyse (Optimiert für Batch-Prozesse)
def analyze_sentiment_batch(texts, batch_size=32):
    """Effiziente GPU-gestützte Sentiment-Analyse mit CryptoBERT für eine Liste von Texten."""
    results = []

    # Ersetze leere Einträge durch "neutral"
    texts = [t if isinstance(t, str) and t.strip() != "" else "neutral" for t in texts]

    # Batchweise Verarbeitung
    for i in tqdm(range(0, len(texts), batch_size), desc="🔍 Analysiere Sentiments"):
        batch_texts = texts[i : i + batch_size]

        # Tokenisierung (mit Padding für Performance)
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)

        # Vorhersage mit CryptoBERT
        with torch.no_grad():
            outputs = model(**inputs)

        scores = F.softmax(outputs.logits, dim=1)
        labels = ["bearish", "neutral", "bullish"] 

        # Ergebnisse speichern
        for i in range(len(batch_texts)):
            sentiment = labels[torch.argmax(scores[i]).item()]
            confidence = scores[i].max().item()
            results.append((sentiment, confidence))

    return results

# 🔹 Sentiment für **Posts** berechnen
tqdm.pandas()  # Fortschrittsanzeige aktivieren
df_posts["full_text"] = df_posts["title"] + " " + df_posts["selftext"].fillna("")
df_posts[["sentiment", "sentiment_confidence"]] = pd.DataFrame(
    analyze_sentiment_batch(df_posts["full_text"].tolist()), index=df_posts.index
)

# 🔹 Sentiment für **Kommentare** berechnen
df_comments["full_text"] = df_comments["selftext"].fillna("")
df_comments[["sentiment", "sentiment_confidence"]] = pd.DataFrame(
    analyze_sentiment_batch(df_comments["full_text"].tolist()), index=df_comments.index
)

# 🔹 Ergebnisse anzeigen
print(f"✅ Sentiment-Analyse abgeschlossen: {len(df_posts)} Posts & {len(df_comments)} Kommentare bewertet.")


🚀 Verwende Gerät: cuda


🔍 Analysiere Sentiments: 100%|██████████| 5264/5264 [21:59<00:00,  3.99it/s]

✅ Sentiment-Analyse abgeschlossen: 4767 Posts & 168431 Kommentare bewertet.


## Clean

In [9]:
def clean_data(df_posts, df_comments, comment_threshold=500):
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])

    # 2. Fehlende Werte behandeln
    df_posts = df_posts.fillna('')
    df_comments = df_comments.fillna('')

    # 3. Entferne unnötige Spalten (title & selftext)
    df_posts = df_posts.drop(columns=["title", "selftext"], errors="ignore")
    df_comments = df_comments.drop(columns=["selftext"], errors="ignore")

    # 4. Entferne Nutzer mit übermäßigen Kommentaren
    comment_counts = df_comments["author"].value_counts()
    frequent_users = comment_counts[comment_counts > comment_threshold].index
    df_comments = df_comments[~df_comments["author"].isin(frequent_users)]

    print(f"✅ Daten bereinigt: {df_comments.shape[0]} Kommentare übrig (nach Spam-Filter).")

    return df_posts, df_comments


In [10]:
df_posts_clean, df_comments_clean = clean_data(df_posts , df_comments, comment_threshold=500)

✅ Daten bereinigt: 144189 Kommentare übrig (nach Spam-Filter).


## Merge

In [11]:
posts = df_posts_clean.copy()
comments = df_comments_clean.copy()

In [12]:
# 🔹 **Relevante Spalten für den Merge**
posts = posts[["post_id", "crypto", "search_term", "subreddit", "author", "date", "time", "score", "full_text", "sentiment", "sentiment_confidence"]]
comments = comments[["post_id", "comment_id", "author", "date", "time", "score", "full_text", "sentiment", "sentiment_confidence"]]

# 🔹 **Kommentare erben `crypto`, `search_term` und `subreddit` vom Post**
comments = comments.merge(df_posts[["post_id", "crypto", "search_term", "subreddit"]], on="post_id", how="left")

# 🔹 `type`-Spalte für Unterscheidung hinzufügen
posts["comment_id"] = None  # Posts haben keine comment_id
posts["type"] = "post"
comments["type"] = "comment"

# 🔹 **Gemeinsame Spalten für den Merge**
common_columns = [
    "post_id", "comment_id", "type", "date", "time", "crypto",
    "search_term", "subreddit", "author","full_text","score", "sentiment", "sentiment_confidence",
]

# 🔹 **Merging der Daten (Posts + Kommentare)**
df_merged = pd.concat([posts[common_columns], comments[common_columns]], ignore_index=True)

# 🔍 Debugging: Überprüfung der Größe
print(f"📌 Merged Dataset: {df_merged.shape[0]} Einträge (Posts + Kommentare)")

# 🔹 Überprüfen, ob alles korrekt normalisiert wurde
print(df_merged.head())

📌 Merged Dataset: 148956 Einträge (Posts + Kommentare)
   post_id comment_id  type        date      time   crypto search_term  \
0  1iylv7t       None  post  2025-02-26  11:58:15  Bitcoin     bitcoin   
1  1iyklm7       None  post  2025-02-26  10:33:56  Bitcoin     bitcoin   
2  1iyfuac       None  post  2025-02-26  05:03:15  Bitcoin     bitcoin   
3  1iyc3gc       None  post  2025-02-26  01:45:40  Bitcoin     bitcoin   
4  1iy7qz7       None  post  2025-02-25  22:25:23  Bitcoin     bitcoin   

        subreddit              author  \
0  CryptoCurrency             IDGAFOS   
1  CryptoCurrency           kirtash93   
2  CryptoCurrency      Odd-Radio-8500   
3  CryptoCurrency            eman2top   
4  CryptoCurrency  Illperformance6969   

                                           full_text  score sentiment  \
0  With AI here, the 4 year cycle will become obs...      0   bearish   
1  China's Supreme People's Court Met To Discuss ...    159   neutral   
2  Cathie Wood Says Bitcoin Consol

In [13]:
df_posts_clean.head()

,post_id,crypto,search_term,subreddit,author,date,time,score,num_comments,full_text,sentiment,sentiment_confidence
0,1iylv7t,Bitcoin,bitcoin,CryptoCurrency,IDGAFOS,2025-02-26,11:58:15,0,11,"With AI here, the 4 year cycle will become obs...",bearish,0.480304
1,1iyklm7,Bitcoin,bitcoin,CryptoCurrency,kirtash93,2025-02-26,10:33:56,159,59,China's Supreme People's Court Met To Discuss ...,neutral,0.560484
2,1iyfuac,Bitcoin,bitcoin,CryptoCurrency,Odd-Radio-8500,2025-02-26,05:03:15,128,35,Cathie Wood Says Bitcoin Consolidation Extreme...,neutral,0.722356
3,1iyc3gc,Bitcoin,bitcoin,CryptoCurrency,eman2top,2025-02-26,01:45:40,0,19,Could Jack Dorsey actually be Satoshi Nakamoto?,bullish,0.724085
4,1iy7qz7,Bitcoin,bitcoin,CryptoCurrency,Illperformance6969,2025-02-25,22:25:23,29,23,Steady Lads: Bitcoin Still On Track to Hit $20...,neutral,0.674270


In [14]:
df_comments_clean.head()

,post_id,comment_id,author,date,time,score,full_text,sentiment,sentiment_confidence
0,1iylv7t,mevdzki,MoonsModBot,2025-02-26,11:58:19,1,\nStop overpaying on crypto Fees! Buy crypto w...,neutral,0.682685
1,1iylv7t,mevedg4,graytleapforward,2025-02-26,12:01:20,8,Is that you Siri?,neutral,0.500153
2,1iylv7t,mevl2uw,GreedVault,2025-02-26,12:49:29,3,My AI agent will compete with yours for limite...,neutral,0.581301
3,1iylv7t,mevmt8r,inShambles3749,2025-02-26,13:00:55,2,Has literally nothing to do with AI but whatever,neutral,0.737131
4,1iylv7t,mevoed8,Lagna85,2025-02-26,13:11:07,2,Algorithm trading bots have existed decade ago,neutral,0.679802


In [15]:
df_merged.head()

,post_id,comment_id,type,date,time,crypto,search_term,subreddit,author,full_text,score,sentiment,sentiment_confidence
0,1iylv7t,None,post,2025-02-26,11:58:15,Bitcoin,bitcoin,CryptoCurrency,IDGAFOS,"With AI here, the 4 year cycle will become obs...",0,bearish,0.480304
1,1iyklm7,None,post,2025-02-26,10:33:56,Bitcoin,bitcoin,CryptoCurrency,kirtash93,China's Supreme People's Court Met To Discuss ...,159,neutral,0.560484
2,1iyfuac,None,post,2025-02-26,05:03:15,Bitcoin,bitcoin,CryptoCurrency,Odd-Radio-8500,Cathie Wood Says Bitcoin Consolidation Extreme...,128,neutral,0.722356
3,1iyc3gc,None,post,2025-02-26,01:45:40,Bitcoin,bitcoin,CryptoCurrency,eman2top,Could Jack Dorsey actually be Satoshi Nakamoto?,0,bullish,0.724085
4,1iy7qz7,None,post,2025-02-25,22:25:23,Bitcoin,bitcoin,CryptoCurrency,Illperformance6969,Steady Lads: Bitcoin Still On Track to Hit $20...,29,neutral,0.674270


## Export 

In [16]:
# Setze den Pfad zu deinem Google Drive Ordner
DRIVE_PATH = "G:/Meine Ablage/reddit/"
POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts.csv")
COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments.csv")
MERGED_CSV = os.path.join(DRIVE_PATH, "reddit_merged.csv")
ORIGINAL_POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts_original.csv")
ORIGINAL_COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments_original.csv")

Funktion zum Export 

In [17]:
def append_to_csv(df_new, filename, key_column):
    """Hängt neue Daten an eine bestehende CSV an & entfernt Duplikate."""
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls Datei existiert, alte Daten einlesen
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep="|", encoding="utf-8-sig", on_bad_lines="skip")
            
            # 🔹 Daten zusammenführen & Duplikate nach `key_column` entfernen (neuere Werte behalten)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True).drop_duplicates(subset=[key_column], keep="last")
        else:
            df_combined = df_new  # Falls keine Datei existiert, neue Daten direkt nutzen

        # 🔹 CSV speichern
        df_combined.to_csv(
            file_path,
            index=False,
            sep="|",
            encoding="utf-8-sig",
            lineterminator="\n"
        )
        print(f"✅ Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts_clean, df_comments_clean, df_merged,df_posts, df_comments):
    """Speichert Posts, Kommentare & die gemergte Datei mit Duplikat-Prüfung."""
    try:
        append_to_csv(df_posts_clean, "reddit_posts.csv", key_column="post_id")
        append_to_csv(df_comments_clean, "reddit_comments.csv", key_column="comment_id")
        append_to_csv(df_merged, "reddit_merged.csv", key_column="comment_id")  # Falls Kommentare entscheidend sind
        append_to_csv(df_posts, "reddit_posts_original.csv", key_column="post_id")
        append_to_csv(df_comments, "reddit_comments_original.csv", key_column="comment_id")
    except Exception as e:
        print(f"Fehler beim Export: {e}")

In [19]:
# Export-Funktion aufrufen
export_to_drive(df_posts_clean, df_comments_clean, df_merged,df_posts, df_comments)

✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_posts.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_comments.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_merged.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_posts_original.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_comments_original.csv
